In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
from tensorflow.python.keras import layers
import tensorflow_probability as tfp
from tensorflow.python.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

import keras.backend as K


import functions.parse_data as parse
import functions.models as md
import functions.handy_functions as hf

from keras.utils import plot_model
from IPython.display import Image

from tensorflow_probability import layers as tfpl

In [2]:
data_water=parse.parse('cloudrm_water.dat')
data_clear=parse.parse('cloudrm_clear.dat')
data_ice=parse.parse('cloudrm_ice.dat')
data_mixed=parse.parse('cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])


In [3]:
#Function taken from: https://github.com/sachinruk/KerasQuantileModel/blob/master/Keras%20Quantile%20Model.ipynb
#As a notebook from reading: https://towardsdatascience.com/deep-quantile-regression-c85481548b5a
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [3]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle','Cloud_Type','Profile_ID','GOT','Water_Vapor']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.

y_labels=['COT']

df=hf.normalise_input_df(data_all,X_labels)
df=hf.add_noise(df,X_labels,sigma=0.001)
num_epochs=2
batch_size=100


##Split data##
X=df[X_labels]
y=df[y_labels]

X_train, y_train, X_val, y_val, X_test, y_test=hf.split_data(X,y,split=[0.9,0.05,0.05])







Epoch 1/2

1800/1800 [==============================] - 55s 30ms/step - loss: 35.7907 - mse: 35.7907 - val_loss: 11.5850 - val_mse: 11.5850
Epoch 2/2
1800/1800 [==============================] - 54s 30ms/step - loss: 12.6319 - mse: 12.6319 - val_loss: 10.5552 - val_mse: 10.5552


In [ ]:
qs=[0.1,0.5,0.9]
for q in qs:

    ##Create model##
    model=tf.keras.Sequential([
        layers.Dense(32,input_dim=len(X_labels),activation='linear'),
        layers.Dense(64,activation='relu'),
        layers.Dense(64,activation='relu'),
        layers.Dense(64,activation='relu'),
        layers.Dense(len(y_labels),activation='linear')
    ])
    ##Compile model##
    model.compile(
        optimizer="adam",
        loss=lambda y,f: tilted_loss(q,y,f),
        metrics=["mse"],
        run_eagerly=True
    )

    ##Train model##
    history=md.LossHistory()
    model.fit(X_train,y_train,
            epochs=num_epochs,
            validation_data=(X_val,y_val),
            batch_size=batch_size,
            callbacks=[history])

In [4]:
test_loss, test_mse = model.evaluate(X_test, y_test, verbose=2)
print(f"Test mse: {test_mse}")

594/594 - 5s - loss: 10.3719 - mse: 10.3719 - 5s/epoch - 9ms/step
Test mse: 10.371944427490234


In [5]:
y_predict=model.predict(X_test)


594/594 [==============================] - 2s 4ms/step


Simultaneous multiple quantiles instead of a new network for each quantile may be found in:
https://github.com/strongio/quantile-regression-tensorflow/blob/master/Quantile%20Loss.ipynb

In [13]:
from typhon.retrieval.qrnn.qrnn import QRNN

In [63]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle','Cloud_Type','Profile_ID','GOT','Water_Vapor']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.

y_labels=['COT']

df=hf.normalise_input_df(data_all,X_labels)
df=hf.add_noise(df,X_labels,sigma=0.001)
num_epochs=2
batch_size=100


##Split data##
X=df[X_labels]
y=df[y_labels]

X_train, y_train, X_val, y_val, X_test, y_test=hf.split_data(X,y,split=[0.9,0.05,0.05])

model=tf.keras.Sequential([
    layers.Dense(32,input_dim=len(X_labels),activation='linear'),
    layers.Dense(64,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(len(y_labels),activation='linear')
])


In [78]:
quantiles=[0.1,0.5,0.9]
qrnn=QRNN(len(X_train.columns),quantiles=quantiles,model=(3,128,'relu'))

train_data_tuple=(X_train.values,y_train.values)
val_data_tuple=(X_val.values,y_val.values)

qrnn.train(training_data=train_data_tuple,
            initial_learning_rate = 0.01,
            learning_rate_minimum = 1e-4,
            batch_size=100,
            convergence_epochs=10,
            maximum_epochs=100,
            training_split=0.05)


Epoch 1/100


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\typhon\retrieval\qrnn\models\keras.py:448: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.fit_generator(


1800/1800 [==============================] - 4s 2ms/step - loss: 2.6062
Epoch 2/100
1800/1800 [==============================] - 3s 2ms/step - loss: 2.3719
Epoch 3/100
1800/1800 [==============================] - 3s 2ms/step - loss: 2.2569
Epoch 4/100
1800/1800 [==============================] - 3s 2ms/step - loss: 2.1708
Epoch 5/100
1800/1800 [==============================] - 3s 2ms/step - loss: 2.0850
Epoch 6/100
1800/1800 [==============================] - 3s 2ms/step - loss: 2.0112
Epoch 7/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.9357
Epoch 8/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.8643
Epoch 9/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.8038
Epoch 10/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.7444
Epoch 11/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.7098
Epoch 12/100
1800/1800 [==============================] - 3s 2ms/step - loss: 1.6392
E

In [91]:
qrnn.predict(X_test.values) # Why is this not working??

ValueError: in user code:

    File "c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non